
# KDD Cup 1999 - Anomali Tespiti

Bu defter, KDD Cup 1999 veri seti üzerinde **anomali (saldırı) tespiti** için uçtan uca bir akış sunar:
1. **Veri Yükleme** (CSV)
2. **Ön İşleme** (One-Hot + Standardizasyon, Train/Test)
3. **Modeller**: Lojistik Regresyon, Random Forest, Decision Tree, Lineer (SGDClassifier), *(opsiyonel)* XGBoost
4. **Hiperparametre Optimizasyonu** (GridSearchCV)
5. **Değerlendirme** (ROC-AUC, PR-Eğrisi, F1, Confusion Matrix)
6. **PCA(2)** üzerinde **karar sınırı** görselleştirme (SVM ile)
7. **K-Means**: Elbow (WCSS) + **Silhouette** skoru
8. **SelectKBest (ANOVA F)** ile örnek **özellik seçimi**

> Bu çalışma **sınıflandırma** problemidir (Normal vs Attack).



In [3]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    roc_auc_score, roc_curve, precision_recall_curve,
    average_precision_score, f1_score, confusion_matrix,
    classification_report
)
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier


try:
    from xgboost import XGBClassifier
    XGB_AVAILABLE = True
except Exception as e:
    XGB_AVAILABLE = False
    print("XGBoost bulunamadı. Kurmak için: pip install xgboost")

from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_selection import SelectKBest, f_classif

import joblib

plt.rcParams['figure.figsize'] = (8, 6)
RANDOM_STATE = 42



## I. Veri Setinin Yüklenmesi

Bu bölümde **KDD Cup 1999** veri seti `pandas` ile CSV'den okunur.  
Etiket sütunu (`label`) **ikili** hale getirilir: `normal` → 0, diğer saldırılar → 1.


In [4]:

# CSV dosya yolu
DATA_PATH = "kdd99.csv"

# KDD 1999 özellik isimleri (41 + label)
KDD_COLUMNS = [
    "duration","protocol_type","service","flag","src_bytes","dst_bytes","land",
    "wrong_fragment","urgent","hot","num_failed_logins","logged_in","num_compromised",
    "root_shell","su_attempted","num_root","num_file_creations","num_shells",
    "num_access_files","num_outbound_cmds","is_host_login","is_guest_login",
    "count","srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate",
    "same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count",
    "dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate",
    "dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
    "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate",
    "label"
]

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(
        f"Veri dosyası bulunamadı: {DATA_PATH}\n"
        "CSV dosyasını klasöre koyup DATA_PATH'i güncelleyin."
    )

df = pd.read_csv(DATA_PATH, header=None, names=KDD_COLUMNS)

# İkili hedef
df['label_binary'] = np.where(df['label'].astype(str).str.contains('normal'), 0, 1)

print("Veri şekli:", df.shape)
print("Saldırı oranı (1):", round(df['label_binary'].mean(), 4))
df.head()


FileNotFoundError: Veri dosyası bulunamadı: kdd99.csv
CSV dosyasını klasöre koyup DATA_PATH'i güncelleyin.


## II. Veri Ön İşleme

- Kategorik: `protocol_type`, `service`, `flag` → **One-Hot Encoding**  
- Sayısal: Diğer tüm sütunlar → **StandardScaler**  
- **Stratified** train/test bölme (sınıf oranını korur)


In [ ]:

categorical_cols = ['protocol_type', 'service', 'flag']
numeric_cols = [c for c in df.columns if c not in categorical_cols + ['label', 'label_binary']]

X = df[categorical_cols + numeric_cols]
y = df['label_binary']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", StandardScaler(with_mean=False), numeric_cols)  # sparse uyumu
    ]
)

# Dengesizlik için ölçek önerisi (XGBoost'ta kullanılacak)
pos = (y_train == 1).sum()
neg = (y_train == 0).sum()
scale_pos_weight_val = float(neg) / float(pos) if pos > 0 else 1.0
print("scale_pos_weight önerisi:", round(scale_pos_weight_val, 2))



## III. Modeller

Aşağıdaki klasik algoritmalar bir **Pipeline** içinde eğitilecektir:
- **Lojistik Regresyon** (baseline)
- **Random Forest**
- **Decision Tree**
- **SGDClassifier** (lineer, `loss='log_loss'`)  
- *(Opsiyonel)* **XGBoost** (varsa)


In [ ]:

logreg_pipe = Pipeline([("prep", preprocess),
                        ("clf", LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))])

rf_pipe = Pipeline([("prep", preprocess),
                    ("clf", RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1))])

dt_pipe = Pipeline([("prep", preprocess),
                    ("clf", DecisionTreeClassifier(random_state=RANDOM_STATE))])

sgd_pipe = Pipeline([("prep", preprocess),
                     ("clf", SGDClassifier(loss="log_loss", class_weight="balanced", random_state=RANDOM_STATE))])

if XGB_AVAILABLE:
    xgb_pipe = Pipeline([("prep", preprocess),
                         ("clf", XGBClassifier(
                             objective="binary:logistic",
                             eval_metric="logloss",
                             tree_method="hist",
                             random_state=RANDOM_STATE,
                             n_jobs=-1
                         ))])



## IV. Hiperparametre Optimizasyonu (GridSearchCV)

Her model için **ROC-AUC** puanını maksimize edecek şekilde arama yapılır.


In [ ]:

search_spaces = {
    "Logistic Regression": (logreg_pipe, {
        "clf__C": [0.1, 1.0, 3.0],
        "clf__penalty": ["l2"],
        "clf__solver": ["lbfgs", "liblinear"]
    }),
    "Random Forest": (rf_pipe, {
        "clf__n_estimators": [100, 200],
        "clf__max_depth": [None, 20, 40],
        "clf__min_samples_split": [2, 5],
        "clf__min_samples_leaf": [1, 2]
    }),
    "Decision Tree": (dt_pipe, {
        "clf__max_depth": [None, 10, 20, 40],
        "clf__min_samples_split": [2, 5, 10],
        "clf__min_samples_leaf": [1, 2, 4],
        "clf__class_weight": [None, "balanced"]
    }),
    "SGD (Linear)": (sgd_pipe, {
        "clf__alpha": [1e-4, 1e-3, 1e-2],
        "clf__max_iter": [1000, 2000],
        "clf__tol": [1e-3, 1e-4]
    })
}

if XGB_AVAILABLE:
    search_spaces["XGBoost"] = (xgb_pipe, {
        "clf__n_estimators": [200, 400],
        "clf__max_depth": [4, 6, 8],
        "clf__learning_rate": [0.03, 0.1],
        "clf__subsample": [0.8, 1.0],
        "clf__colsample_bytree": [0.8, 1.0],
        "clf__scale_pos_weight": [1.0, scale_pos_weight_val]
    })

best_models = {}
cv_results = []

for name, (pipe, grid) in search_spaces.items():
    print(f"\n>> {name} GridSearch başlıyor...")
    gs = GridSearchCV(
        estimator=pipe,
        param_grid=grid,
        scoring="roc_auc",
        cv=3,
        n_jobs=-1,
        verbose=1
    )
    gs.fit(X_train, y_train)
    best_models[name] = gs.best_estimator_
    cv_results.append((name, gs.best_score_, gs.best_params_))
    print(f"{name} en iyi ROC-AUC (CV): {gs.best_score_:.4f}")
    print(f"{name} en iyi parametreler: {gs.best_params_}")

cv_summary = pd.DataFrame(cv_results, columns=["Model", "ROC-AUC (CV)", "Best Params"]).sort_values("ROC-AUC (CV)", ascending=False)
cv_summary



## V. Model Değerlendirme

Her model için test setinde:
- **ROC-AUC**, **PR-AUC**, **F1** skorları
- **ROC** ve **Precision-Recall** eğrileri
- **Karmaşıklık Matrisi (Confusion Matrix)**


In [ ]:

def evaluate_classifier(name, model, X_tr, y_tr, X_te, y_te):
    # Olasılıklar
    y_tr_proba = model.predict_proba(X_tr)[:, 1]
    y_te_proba = model.predict_proba(X_te)[:, 1]

    # 0.5 eşik ile sınıflar
    y_tr_pred = (y_tr_proba >= 0.5).astype(int)
    y_te_pred = (y_te_proba >= 0.5).astype(int)

    # Skorlar
    roc_auc_tr = roc_auc_score(y_tr, y_tr_proba)
    roc_auc_te = roc_auc_score(y_te, y_te_proba)
    ap_tr = average_precision_score(y_tr, y_tr_proba)
    ap_te = average_precision_score(y_te, y_te_proba)
    f1_tr = f1_score(y_tr, y_tr_pred)
    f1_te = f1_score(y_te, y_te_pred)

    print(f"\n[{name}]")
    print("ROC-AUC  (train/test):", round(roc_auc_tr,4), "/", round(roc_auc_te,4))
    print("PR-AUC   (train/test):", round(ap_tr,4), "/", round(ap_te,4))
    print("F1-score (train/test):", round(f1_tr,4), "/", round(f1_te,4))
    print("\nClassification Report (Test):\n", classification_report(y_te, y_te_pred, digits=4))

    # Confusion Matrix
    cm = confusion_matrix(y_te, y_te_pred)
    plt.figure()
    plt.imshow(cm, interpolation='nearest')
    plt.title(f'Confusion Matrix - {name}')
    plt.colorbar()
    plt.xticks([0,1], ['Normal (0)', 'Attack (1)'])
    plt.yticks([0,1], ['Normal (0)', 'Attack (1)'])
    plt.xlabel('Predicted'); plt.ylabel('True')
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, cm[i, j], ha='center', va='center')
    plt.tight_layout(); plt.show()

    # ROC
    fpr, tpr, _ = roc_curve(y_te, y_te_proba)
    plt.figure()
    plt.plot(fpr, tpr, label=f'{name}')
    plt.plot([0,1],[0,1],'--')
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {name}'); plt.legend(); plt.tight_layout(); plt.show()

    # PR
    prec, rec, _ = precision_recall_curve(y_te, y_te_proba)
    plt.figure()
    plt.plot(rec, prec, label=f'{name}')
    plt.xlabel('Recall'); plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - {name}'); plt.legend(); plt.tight_layout(); plt.show()

for name, model in best_models.items():
    evaluate_classifier(name, model, X_train, y_train, X_test, y_test)



## VI. Özellik Önemi (Random Forest) ve Eşik Ayarı (Opsiyonel)

- **Özellik önemi**: En anlamlı değişkenleri görmek için.  
- **Eşik ayarı**: İhtiyaca göre **precision/recall** dengesini değiştirmek için.


In [ ]:

# RF bulunursa önemleri çıkar
if "Random Forest" in best_models:
    best_rf = best_models["Random Forest"]
    ohe = best_rf.named_steps['prep'].named_transformers_['cat']
    cat_feature_names = list(ohe.get_feature_names_out(['protocol_type','service','flag']))
    num_feature_names = [c for c in df.columns if c not in ['protocol_type','service','flag','label','label_binary']]
    all_feature_names = cat_feature_names + num_feature_names

    rf = best_rf.named_steps['clf']
    importances = rf.feature_importances_
    feat_imp = pd.DataFrame({"feature": all_feature_names, "importance": importances})\
               .sort_values("importance", ascending=False).head(20)
    display(feat_imp)

    plt.figure()
    plt.barh(feat_imp['feature'][::-1], feat_imp['importance'][::-1])
    plt.xlabel("Importance"); plt.title("Top 20 Feature Importances (Random Forest)")
    plt.tight_layout(); plt.show()

    # Eşik Ayarı (RF)
    y_proba = best_rf.predict_proba(X_test)[:, 1]
    thresholds = np.linspace(0.1, 0.9, 9)
    rows = []
    for th in thresholds:
        y_pred = (y_proba >= th).astype(int)
        tp = ((y_pred == 1) & (y_test == 1)).sum()
        fp = ((y_pred == 1) & (y_test == 0)).sum()
        fn = ((y_pred == 0) & (y_test == 1)).sum()
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        rows.append({"threshold": th, "precision": precision, "recall": recall, "f1": f1_score(y_test, y_pred)})
    th_df = pd.DataFrame(rows).sort_values("f1", ascending=False)
    display(th_df.head())



## VII. PCA(2D) Üzerinde Sınıf Karar Sınırları (SVM ile)

Sunumlarda sınıfları **X–Y düzleminde görmek** için, eğitim/test verisini pipeline sonrası dönüştürüp **PCA(2)** ile indiriyoruz; ardından bir **SVM** ile karar sınırını çiziyoruz.


In [ ]:

# Pipeline sonrası dönüştür
any_model = list(best_models.values())[0]  # herhangi bir en iyi modelin prep'ini kullan
prep = any_model.named_steps['prep']
X_train_tr = prep.transform(X_train)
X_test_tr  = prep.transform(X_test)

# Dense'e çevir (gerekirse)
X_train_dense = X_train_tr.toarray() if hasattr(X_train_tr, "toarray") else X_train_tr
X_test_dense  = X_test_tr.toarray() if hasattr(X_test_tr, "toarray") else X_test_tr

# PCA(2)
pca_vis = PCA(n_components=2, random_state=RANDOM_STATE)
X_train_2d = pca_vis.fit_transform(X_train_dense)
X_test_2d  = pca_vis.transform(X_test_dense)

# Görsel sınırlayıcı model
svm_vis = SVC(kernel="rbf", probability=True, random_state=RANDOM_STATE)
svm_vis.fit(X_train_2d, y_train)

# Meshgrid
x1_min, x1_max = X_test_2d[:,0].min()-1, X_test_2d[:,0].max()+1
x2_min, x2_max = X_test_2d[:,1].min()-1, X_test_2d[:,1].max()+1
xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, 0.02),
                       np.arange(x2_min, x2_max, 0.02))
Z = svm_vis.predict(np.c_[xx1.ravel(), xx2.ravel()]).reshape(xx1.shape)

plt.figure(figsize=(7,6))
plt.contourf(xx1, xx2, Z, alpha=0.5)
plt.scatter(X_test_2d[y_test==0,0], X_test_2d[y_test==0,1], s=20, label="Normal (0)")
plt.scatter(X_test_2d[y_test==1,0], X_test_2d[y_test==1,1], s=20, label="Attack (1)")
plt.title("PCA(2D) Karar Sınırları (SVM-RBF)")
plt.xlabel("PC1"); plt.ylabel("PC2"); plt.legend(); plt.tight_layout(); plt.show()



## VIII. K-Means: Elbow ve Silhouette

Denetimsiz öğrenmeye örnek olarak, pipeline sonrası dönüştürülmüş özellikler üzerinde **K-Means** için **Elbow (WCSS)** grafiği ve **Silhouette** skorunu hesaplıyoruz.


In [ ]:

X_all = df[categorical_cols + numeric_cols]
X_all_tr = prep.transform(X_all)
X_all_dense = X_all_tr.toarray() if hasattr(X_all_tr, "toarray") else X_all_tr

# Elbow (WCSS)
wcss = []
K = range(2, 11)
for k in K:
    km = KMeans(n_clusters=k, init="k-means++", n_init=10, random_state=RANDOM_STATE)
    km.fit(X_all_dense)
    wcss.append(km.inertia_)

plt.figure()
plt.plot(list(K), wcss, marker="o")
plt.title("K-Means Elbow (Ön İşlem Sonrası Özellikler)")
plt.xlabel("Küme sayısı (k)"); plt.ylabel("WCSS (inertia)")
plt.tight_layout(); plt.show()

# Örnek bir k için silhouette
k_best = 5  # elbow grafiğine göre güncelleyebilirsiniz
km_best = KMeans(n_clusters=k_best, init="k-means++", n_init=10, random_state=RANDOM_STATE)
labels = km_best.fit_predict(X_all_dense)
print("Silhouette skoru:", round(silhouette_score(X_all_dense, labels), 3))



## IX. SelectKBest (ANOVA F) ile Örnek Özellik Seçimi

Karma veri tiplerinde, yalnızca **sayısal** sütunlar üzerinde **ANOVA F (f_classif)** ile en iyi `k` özellik seçimi.


In [ ]:

X_num = df[numeric_cols].copy()
y_bin = df["label_binary"].copy()

selector = SelectKBest(score_func=f_classif, k=10)  # en iyi 10 sayısal özellik
X_num_sel = selector.fit_transform(X_num, y_bin)
selected_num_features = np.array(numeric_cols)[selector.get_support()]
print("Seçilen sayısal özellikler:", selected_num_features)



## X. Modellerin Kaydedilmesi

En iyi modeller **joblib** ile diske kaydedilir.


In [ ]:

os.makedirs("models", exist_ok=True)
for name, model in best_models.items():
    safe = name.lower().replace(" ", "_")
    joblib.dump(model, f"models/{safe}.joblib")
print("Modeller kaydedildi: models/ klasöründe.")



## XI. Kapanış

- **Binary (Normal vs Attack)** yaklaşımı ile birden çok klasik algoritma karşılaştırıldı.
- **RF** genellikle güçlü sonuç verir; ancak veri yapısına göre diğerleri tercih edilebilir.
- **PCA(2)** ile karar sınırı görselleştirme sunumda etkilidir.
- **K-Means** ve **SelectKBest** ile denetimsiz öğrenme ve özellik seçimine dair kısa demolar eklendi.
- Dengesiz veri için **class_weight**/**scale_pos_weight** gibi yaklaşımlar kritik olabilir.
